![Embeddings](./images/AIStylist.png)
# AI Stylist - Conversational Shopping Experience with Amazon Bedrock


In this notebook, we will build an AI virtual assistant acting as your stylist. You will learn how to build an application that helps users choose clothes from a product catalog, all powered by Large Language Models (LLMs). We will leverage Amazon Bedrock for using the Foundation Models (FMs).

### Context
Virtual Assistants are becoming increasingly popular as LLMs start to power the interaction with users. These applications demand an ability to `understand` the intent of the user input and generate adaptable sequence of calls to language models and various utilities depending on user input.

### Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the customer experience. Virtual assistants use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.

### Pattern
We can improve upon this process by implementing an architecture called `Retreival Augmented Generation (RAG)`. RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. Virtual assistants need ability to determine the sequence of events needed to generate the final results and as such we build towards the use, `plan-and-execute` agents along with `Zero-shot ReAct` which is an action agent and uses [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) pattern to select the appropriate calls. These will then develop into `Agents` concept. The Agent interface enables such flexibility for these applications.

There are two primary categories of agents:

- Action agents: At each interval, determine the subsequent action utilizing the outputs of all previous actions. 
- Plan-and-execute agents: Determine the complete order of actions initially, then implement them all without updating the plan.

### Challenges
- Parse the user query and extracting information which can be used to search the catalog
- Reform the prompts to bring products from the catalog
- Search product catalog to bring relevant products with details like product id, description etc
- Generate relevant images for a style based on the user chat
- Continue the shopping experience workflow for adding products or finalizing the sale
- Identify relevant products based on user location or questions asked by user

### Proposal
To the above challenges, this notebook proposes the following strategy

#### Prepare documents
![Embeddings](./images/embeddings_lang.png)

Before being able to answer the questions, the product catalog must be processed and a stored in a document store index
- Load the product catalog and specifically the descriptions
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings

#### Catalog operations. Fetch relevant products from the catalog
We need to leverage the LLM for the following 
- Extract information of the `intent` and other details like `place` , objective of the purchase
- Use [LangChain](https://python.langchain.com/docs/get_started/introduction) as an orchestrator to do a similarity search from the vector store and bring back the artifacts matching the query
- Extract information for `Product ID` from the search results  and conduct a search to bring further details of the selected products
- Leverage [LangChain](https://python.langchain.com/docs/get_started/introduction) to orchestrate and call API's using the `ReAct` framework to bring other details like weather and conduct search to bring back the relevant products

#### Query
- Identify the `intent` of the query
- Orchestrate the workflow to move the users to the next steps in their purchase journey

![Question](./images/chatbot_lang.png)

After the document index is prepared, you can ask questions and related documents will be fetched. The following steps will be executed.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved


## Virtual Assistant using Amazon Bedrock

![Amazon Bedrock - Conversational Interface](./images/context-aware-chatbot.png)

## Langchain framework for building Virtual Assistants with Amazon Bedrock
In conversational interfaces such as virtual assistants, it is highly important to remember previous interactions, both at a short term but also at a long term level.

LangChain provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular and useful regardless of how they are used. Secondly, LangChain is able to retrieve previous conversations end other context fromvector stores.
It allows us to easily define and interact with different types of abstractions, which make it easy to build powerful chatbots.

## Building Chatbot with Context - Key Elements

The first process in a building a contextual-aware chatbot is to **generate embeddings** for the context. Typically, you will have an ingestion process which will run through your embedding model and generate the embeddings which will be stored in a sort of a vector store. In this example we are using Titan Embeddings model for this

![Embeddings](./images/embeddings_lang.png)

The second process is the user request orchestration, interaction, invoking and returning the results

![Chatbot](./images/chatbot_lang.png)

## Architecture
![Architecture](./images/Architecture.png)

### <font color='red'>Setup</font> 
---
<font color='red'>⚠️ ⚠️ ⚠️</font> 
Before running this notebook, ensure you've run the [Intro to Bedrock notebook](./intro_to_bedrock.ipynb) notebook. <font color='red'>⚠️ ⚠️ ⚠️</font>

---

In [ ]:
!pip install -r requirements.txt

## Configure Bedrock

Create the necessary clients to invoke Bedrock models. If you would need to pass in a certain role then set those values appropriately
We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude V2 for text generation and Titan Embeddings G1 - Text for text embeddings.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="anthropic.claude-v2")`

You can read about all the available model IDs [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
!pip install langchain

In [5]:
import json
import os
from pprint import pprint
import sys
import boto3
import botocore

from langchain.llms.bedrock import Bedrock
from IPython.display import Image

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
# os.environ["AWS_PROFILE"] = ""
# os.environ["BEDROCK_ASSUME_ROLE"] = ""  # E.g. "arn:aws:..."

# Adam: What is the difference between runtime?
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False)

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None))

model_parameter = {
    "temperature": 0.0, 
    "top_p": .5, 
    "top_k": 250, 
    "max_tokens_to_sample": 2000, 
    "stop_sequences": ["\n\n Human: bye"]
}
llm = Bedrock(
    model_id="anthropic.claude-v2", 
    model_kwargs=model_parameter, 
    client=bedrock_runtime
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)
Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Implementation
In order to follow the approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V2 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: [S3FileLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.s3_file.S3FileLoader.html) and PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: In-Memory store FAISS

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

## Simulate a user ask of the AI Stylist 

![Generate Look](./images/user_prompt.png)

### Try these prompts if you would like
- `"I am a female journalist in my 30s traveling to New York next week. What kind of outfit should I wear on my first day at New York Times?"`
- `"I am male athlete looking for suitable outfit for attending half marathon in San Fransisco."`

In [6]:
# If you'd like to try your own prompt, edit this parameter!
customer_input = "I'm a 32-year-old female going on a bachelorette trip to San Juan, Puerto Rico. The weather is supposed to be between 80–90° and sunny. We are going to spend one day in the old city of San Juan and one day at the beach. I'm going to be with five other women my age. What do you recommend I wear?"


# Customer id to infuse order history and delivery address
customer_id = "2"

## Extract `product relevant` information

We have our product catalog stored in a data base and the key attributes for those are 
1. Product type
2. Age group for the product
3. Season for use of the product
4. Description of the product
5. Images for the product

We will instruct LLM to fetch the relevant information from the user prompt based on the above so we can query our catalog and bring back relevant results. Note we have a very specific prompt template for the `entity extraction`

In [7]:
# Identify product attributes from customer prompt to generate better results
ner_prompt = """Human: Find person age group, gender, season and the location in the customer input.
Instructions:
The age group can be one of the following: 10-20, 20-30, 30-50, 50+
The gender can be one of the following: Mens, Womens, Other
The gender can also be derived from the name if not explicitly mentioned
The season can be one of the following: summer, winter, spring, fall
The output must be in JSON format inside the tags <attributes></attributes>

If the information of an entity is not available in the input then don't include that entity in the JSON output

Begin!

Customer input: {customer_input}
Assistant:"""
entity_extraction_result = llm(ner_prompt.format(customer_input=customer_input)).strip()
print(entity_extraction_result)

<attributes>
{
  "age_group": "30-50",
  "gender": "Womens",
  "season": "summer",
  "location": "San Juan, Puerto Rico"
}
</attributes>


#### Extract values into JSON

Since we have instructed LLM to return our data as XML wrapping a JSON, we run the necessary extraction steps to fetch the relevant details.

In [8]:
import re
import json
result = re.search('<attributes>(.*)</attributes>', entity_extraction_result, re.DOTALL)
attributes = json.loads(result.group(1))
attributes

{'age_group': '30-50',
 'gender': 'Womens',
 'season': 'summer',
 'location': 'San Juan, Puerto Rico'}

## Use Retrieval Augmented Generation (RAG) 

We will leverage the semantic search to embed product-accessory catalogs and order history from the embeddings created by `Amazon Titan Embeddings Text v1`

After downloading we can load the documents with the help of [S3FileLoader available under LangChain](https://python.langchain.com/docs/modules/data_connection/document_loaders/) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 8k tokens, which roughly translates to ~32000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

Here we are fetching our product catalog and creating the embeddings for 
1. Customer reviews
2. Order History
3. Product catalog description 

In [9]:
!pip install unstructured

In [21]:
!pip install mkl
!python -m pip uninstall -y numpy
!pip install numpy==1.26.4

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.0 which is incompatible.


In [10]:
np.__version__

'1.26.4'

In [12]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 13.0 MB/s eta 0:00:00:00:010:01m


In [18]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
# from langchain.embeddings import BedrockEmbeddings
from langchain_community.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import S3FileLoader
import numpy as np

In [29]:
# - create the Titan Embeddings Model
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_runtime)
customer_reviews_loader = S3FileLoader("sagemaker-example-files-prod-us-east-1", "datasets/image/howser-bedrock/data/aistylist/data/customer_reviews.csv", region_name="us-east-1",)
order_history_loader = S3FileLoader("sagemaker-example-files-prod-us-east-1", "datasets/image/howser-bedrock/data/aistylist/data/order_history.csv", region_name="us-east-1")
products_catalog_loader = S3FileLoader("sagemaker-example-files-prod-us-east-1", "datasets/image/howser-bedrock/data/aistylist/data/products_catalog.csv", region_name="us-east-1")
documents = []
customer_reviews = customer_reviews_loader.load()
order_history = order_history_loader.load()
products_catalog = products_catalog_loader.load()
products_catalog[0].dict().keys()

dict_keys(['page_content', 'metadata', 'type'])

In [32]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, to test an alternate behavior.
    chunk_size = 1000,
    chunk_overlap  = 100,
)

In [33]:
# Embedding data here. Add more data for better results
documents += text_splitter.split_documents(customer_reviews)
documents += text_splitter.split_documents(order_history)
documents += text_splitter.split_documents(products_catalog)
vectorstore_faiss = FAISS.from_documents(
    documents,
    bedrock_embeddings,
)

In [47]:
pprint(len(documents))
pprint(vectorstore_faiss.embeddings)

183
BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7ff609814970>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None, normalize=False)


In [19]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)
query_embedding = vectorstore_faiss.embedding_function.embed_query(customer_input)
np.array(query_embedding)

array([ 0.19140625, -0.05029297,  0.20214844, ...,  0.17089844,
       -0.6796875 ,  0.21386719])

## Generate *`n`* style recommendations

Make a query to embed the LLM using customer input using LangChain for orchestration of RAG. It also provides a framework for orchestrating RAG flows with purpose built "chains". In this section, we will see how to make a [retrieval chain](https://python.langchain.com/docs/use_cases/question_answering/vector_db_qa) which is more comprehensive and robust than the original retrieval system we built above.

The workflow above uses the following process:
1. User input is received.
2. User input is queried against the vector database to retrieve the relevant products
3. Product description and chat memory are inserted into a new prompt to respond to the user input.
4. This output is fed into the stable diffusion model to return the relevant images

However, more complex methods of interacting with the user input can generate more accurate results in RAG architectures. One of the popular mechanisms which can increase accuracy of these retrieval systems is utilizing more than one call to an LLM in order to reformat the user input for more effective search to your vector database. A better workflow is described below:

1. User input is received.
2. An LLM is used to reword the user input to be a better search query for the vector database based on the chat history and product description. 
3. This could include things like condensing, rewording, addition of chat context, or stylistic changes.
4. Reformatted user input is queried against the vector database to retrieve relevant products. The reformatted user input and relevant documents are inserted into a new prompt in order to generate the new style. 
5. This is then fed into the stable diffusion model to generate the images. 

In your application the images can come from a pre canned images. 

We will now build out this second workflow using LangChain. First we need to make a prompt which will reformat the user input to be more compatible for searching of the vector database. The way we do this is by providing the chat history as well as the some basic instructions to Claude and asking it to condense the input into a single output. 

In [20]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to generate 5 style recommendations for the customer input at the end.
<context>
{context}
</context>
<example>A navy suit with a light blue dress shirt, conservative tie, black oxford shoes, and a leather belt.</example>
<example>A lehenga choli set with a crop top, flowing skirt, and dupatta scarf in lively colors and metallic accents.</example>

Customer Input: {question}
Each style recommendation must be inside the tags <style></style>.
Do not output product physical IDs.
Skip the preamble.
Assistant: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Use RetrievalQA customizations for improving Q&A experience
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 6}
    ),
    return_source_documents=False,
    chain_type_kwargs={"prompt": PROMPT},
)
styles_response = qa({"query": customer_input})['result']

# Alternitively we can query using wrapper also
# styles_response = wrapper_store_faiss.query(question= customer_input, llm=llm)

print_ww(styles_response)

 Here are 5 style recommendations for your bachelorette trip to San Juan:

<style>A strapless maxi dress in a bright, tropical print. Gold jewelry like chandelier earrings,
arm cuffs, and layered necklaces. Flat metallic sandals.</style>

<style>High-waisted denim shorts with a lace bralette or crop top. Woven sandals and a straw hat for
the beach day. Statement sunglasses and tassel earrings.</style>

<style>A long, flowy maxi skirt or dress in a solid color like white or pink. Wedge espadrilles and
a crossbody bag. Delicate jewelry like a pendant necklace.</style>

<style>A flirty romper or jumpsuit with a deep v-neck and fun patterns. Wedge heels that can go from
day to night. Chunky bangles and hoop earrings.</style>

<style>Palazzo pants or culottes with a crop top or bandeau. Slip-on sneakers or sandals. A kimono
coverup and body chains or lariat necklaces.</style>


### Prepare the received response

Since we have instructed LLM to return our data is returned as XML wrapping a JSON, we run the necessary extraction steps to fetch the relevant details to generate images for each look. 

In [48]:
# Prepare input to fetch images for each look
styles = re.findall('<style>(.*?)</style>', styles_response)
styles

['A strapless maxi dress in a bright, tropical print. Gold jewelry like chandelier earrings, arm cuffs, and layered necklaces. Flat metallic sandals.',
 'High-waisted denim shorts with a lace bralette or crop top. Woven sandals and a straw hat for the beach day. Statement sunglasses and tassel earrings.',
 'A long, flowy maxi skirt or dress in a solid color like white or pink. Wedge espadrilles and a crossbody bag. Delicate jewelry like a pendant necklace.',
 'A flirty romper or jumpsuit with a deep v-neck and fun patterns. Wedge heels that can go from day to night. Chunky bangles and hoop earrings.',
 'Palazzo pants or culottes with a crop top or bandeau. Slip-on sneakers or sandals. A kimono coverup and body chains or lariat necklaces.']

## Generate Images for the relevant style

Generate an image for each look using the `Stable Diffusion` model

![Generate Look](./images/generate_look.png)

In [51]:
from PIL import Image
from IPython import display
from base64 import b64decode
import base64
import io
import json
import os
import sys
import ipywidgets as widgets

# Fetching images for each of style
gender_map = {
    'Womens': 'of a female ',
    'Mens': 'of a male '
}

os.makedirs("data/aistylist", exist_ok=True)
image_strip = ""
for i, style in enumerate(styles):
    request = json.dumps({
        "text_prompts": [
            {"text": f"Full body view {gender_map.get(attributes.get('gender'))}without a face in " + style + "dslr, ultra quality, dof, film grain, Fujifilm XT3, crystal clear, 8K UHD", "weight": 1.0},
            {"text": "poorly rendered", "weight": -1.0}
        ],
        "cfg_scale": 9,
        "seed": 4000,
        "steps": 50,
        "style_preset": "photographic",
    })
    modelId = "stability.stable-diffusion-xl"
    
    response = bedrock_runtime.invoke_model(body=request, modelId=modelId)
    response_body = json.loads(response.get("body").read())
    
    base_64_img_str = response_body["artifacts"][0].get("base64")
    # display.display(display.Image(b64decode(base_64_img_str), width=200))
    image_strip += "<td><img src='data:image/png;base64, "+ base_64_img_str + "'></td>"
    with open(f"data/aistylist/image_{i}.png", "wb") as fh:
            fh.write(base64.decodebytes(bytes(base_64_img_str, "utf-8")))

display.display(display.HTML("<table><tr>" + image_strip +"</tr></table>"))

,,,,


## Enhance user experience with Chatbot

#### Generating detailed overview based on customer reviews of products in catalog 
We have discussed the key building blocks needed for the chatbot application and now we will start to create them. LangChain's [ConversationBufferMemory](https://python.langchain.com/docs/use_cases/question_answering/chat_vector_db) class provides an easy way to capture conversational memory for LLM chat applications. Claude will retrieve context through conversational memory using the prompt template. Note that this time our prompt template includes a {chat_history} variable where our chat history will be included to the prompt.

The prompt template has both conversation memory as well as chat history as inputs along with the human input. Notice how the prompt also instructs Claude to not answer questions which it does not have the context for. This helps reduce hallucinations which is extremely important when creating end user facing applications which need to be factual.


![Architecture](./images/chatbot_products.png)

In [76]:
chat_prompt1 = "Show me specific reviews that talk about the quality of the fabric for the jacket."
chat_prompt2 = "What do people like about the business formal jacket?"

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

chat_history = []
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=vectorstore_faiss.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    get_chat_history=lambda h: h, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# Generate detailed reviews based on customer reviews of specific clothing in product catalog

try:
    chat_res1 = conversation.run({'question': chat_prompt1})
    print_ww(chat_res1)
except ValueError as error:
    if  "AccessDeniedException" in str(error):
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

 Based on the context provided, here are two reviews that specifically mention the quality of the
fabric for the jackets:

Coats, Jackets & Vests
I was looking for a warm winter coat that would keep me comfortable during the cold months. I found
exactly what I needed from this store's great selection of coats, jackets and vests. They have
options for both men and women in various styles and fabrics like wool, down and fleece. The prices
are very reasonable too. I'm happy with my purchase and would definitely shop here again for all my
outerwear needs.

Suiting & Blazers
I was very impressed with the quality and fit of the suits and blazers from this company. The fabric
was high-end and durable, with beautiful tailoring that gave a very polished look. The sales staff
was knowledgable and helped me find the perfect suit for an upcoming event. I received many
compliments and felt great wearing it. Overall, an outstanding shopping experience and I would
highly recommend Suiting & Blazers f

In [77]:
try:
    chat_res2 = conversation.run({'question': chat_prompt2 + " Answer even if embeddings does not return anything."})
    print_ww(chat_res2)
except ValueError as error:
    if  "AccessDeniedException" in str(error):
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

 Based on the context provided, it seems people were very impressed with the quality, fit, and
tailoring of the suits and blazers from Suiting & Blazers. The context indicates that the fabric was
high-end and durable, the tailoring gave a very polished look, and the sales staff was
knowledgeable. It also mentions receiving compliments while wearing one of the suits. So it appears
that customers liked the excellent quality, flattering fit, polished tailoring, and overall
outstanding shopping experience when purchasing formalwear from Suiting & Blazers.


#### Customer order history semantic searches

Generating size and color recommendations based on customer's order history. This will help to provide curated content to the customer

In [78]:
chat_prompt3 = "What size and color should I wear?"
chat_res3 = wrapper_store_faiss.query(question= chat_prompt3 + " based on order history for customer with id " + customer_id, llm=llm)
print_ww(chat_res3)

 Based on the order history for customer with id 2, they have previously ordered:

Order 2 - Pant size XS, color black
Order 11 - Pant size M, color black
Order 16 - Jeans size M, color blue

So for this customer, I would recommend getting pants in size M, and the colors black or blue since
those seem to be their preferred sizes and colors for bottoms based on their order history.


## Showing final products based on customer style selection 

Continuing on our architectural pattern we will change the prompt template and leverage the LLM to generate the `recommended` products based on the user selection and weather and other details. The key extraction entities will be 

1. Leverage the customer initial prompt to generate the relevant ids
2. Extract the relevant products from the vector store
3. Physical ID for the products needed



![Architecture](./images/other_products.png)

In [79]:
from PIL import Image
import requests

prompt_template2 = """Human: Extract a list of products and their respective physical IDs from catalog that matches the style given below. 
The catalog of products is provided under <catalog></catalog> tags below.
<catalog>
{context}
</catalog>
Style: {question}

The output should be a JSON of the form <products>[{{"product": <description of the product from the catalog>, "physical_id":<physical id of the product from the catalog>}}, ...]</products>
Skip the preamble.
Assistant: """

PROMPT2 = PromptTemplate(
    template=prompt_template2, input_variables=["context", "question"]
)
qa2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 10}
    ),
    chain_type_kwargs={"prompt": PROMPT2},
    return_source_documents=True,
)

selected_style = styles[3]
print(selected_style)
cart_items = qa2({"query": selected_style })['result']
print_ww(cart_items)

A flirty romper or jumpsuit with a deep v-neck and fun patterns. Wedge heels that can go from day to night. Chunky bangles and hoop earrings.
 Here is the JSON output with products from the catalog matching the given style:

<products>
[
  {"product": "Solid romper with a flattering silhouette. Step into heeled sandals and pick up a
top-handle bag when heading out. Make it pop with a layered necklace and oversized shades.",
"physical_id":"41WeOCW09+L"},

  {"product": "Effortless romper in a vibrant print. Slip on sneakers and grab a leather crossbody
bag for casual days out. Throw on layered necklaces and trendy hoops for an extra touch.",
"physical_id":"41nOYIpZTEL"},

  {"product": "One-piece swimsuit with tropical print and ruffle trim. Cover up with a sheer sarong.
Shield your eyes from the sun with a wide-brim hat and cat-eye sunglasses.",
"physical_id":"41ytDJx3pxL"}
]
</products>


In [81]:
products = json.loads(re.findall('<products>(.*?)</products>', cart_items, re.DOTALL)[0])
products

[{'product': 'Solid romper with a flattering silhouette. Step into heeled sandals and pick up a top-handle bag when heading out. Make it pop with a layered necklace and oversized shades.',
  'physical_id': '41WeOCW09+L'},
 {'product': 'Effortless romper in a vibrant print. Slip on sneakers and grab a leather crossbody bag for casual days out. Throw on layered necklaces and trendy hoops for an extra touch.',
  'physical_id': '41nOYIpZTEL'},
 {'product': 'One-piece swimsuit with tropical print and ruffle trim. Cover up with a sheer sarong. Shield your eyes from the sun with a wide-brim hat and cat-eye sunglasses.',
  'physical_id': '41ytDJx3pxL'}]

In [82]:
from PIL import Image
from IPython import display
import requests
import urllib.parse

cart_item_strip = ""
for product in products:
    url = "https://sagemaker-example-files-prod-us-east-1.s3.us-east-1.amazonaws.com/datasets/image/howser-bedrock/data/aistylist/images/products/" + urllib.parse.quote(product['physical_id'].strip(), safe='', encoding=None, errors=None) + ".jpg"
    # im = Image.open(requests.get(url, stream=True).raw)
    cart_item_strip += "<td><img src='"+ url + "'></td>"
display.display(display.HTML("<table><tr>" + cart_item_strip +"</tr></table>"))

,,


## Integrating DIY Agents to associate external APIs and databases
### Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses. Here we will leverage the user prompt and perform the following actions
1. Extract the city 
2. Get weather information
3. Search our product catalog using semantic search to find relevant products
4. Display the products for user to add to cart

![Architecture](./images/weather.png)

In [83]:
import os
import python_weather

async def getweather(city):
  # declare the client. the measuring unit used defaults to the metric system (celcius, km/h, etc.)
  async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
    # fetch a weather forecast from a city
    weather = await client.get(city)
    
    # returns the current day's forecast temperature (int)
    return weather.current

## Accessory recommendations 

We will provide  accessory recommendations based on location provided in customer input

In [84]:
await getweather(entity_extraction_result[2])

<CurrentForecast temperature=50 description='Light rain shower' kind="Kind.LIGHT_RAIN">

In [85]:
import asyncio
accessory_response = None
if attributes["location"]:
    current_weather = await getweather(entity_extraction_result[2])
    accessory_input = "Suggest list of accessories based on the weather and the selected style. It is " + current_weather.description + " with temperature at " + str(current_weather.temperature) +" degrees fahrenheit.\n Selected Style: " + styles[0]
    accessory_response = qa({"query": accessory_input})['result']
    print_ww(accessory_response)

 <style>
A lightweight trench coat in a neutral color like tan or black to stay dry.
A small crossbody bag or clutch to keep hands free. Waterproof preferred.
Compact umbrella that can easily fit in a bag when not in use.
</style>


In [96]:
if accessory_response:
    accessories = re.findall('<style>(.*?)</style>', accessory_response.strip(), re.DOTALL)
    accessories_items = qa2({"query": ', '.join(accessories)})['result']
    accessories_items = json.loads(re.findall('<products>(.*?)</products>', accessories_items, re.DOTALL)[0])
    accessory_strip = ""
    for accessory in accessories_items:
        url = "https://sagemaker-example-files-prod-us-east-1.s3.us-east-1.amazonaws.com/datasets/image/howser-bedrock/data/aistylist/images/products/" + urllib.parse.quote(accessory['physical_id'].strip(), safe='', encoding=None, errors=None) + ".jpg"
        accessory_strip += "<td><img src='"+ url + "'></td>"
    display.display(display.HTML("<table><tr>" + accessory_strip +"</tr></table>"))

,,


## Simulate the order check out

Add a customer data table to complete the order transaction. This information provides the shipping address for the outfit order.

In [97]:
customer_table=[{"id": 1, "first_name": "John", "last_name": "Doe", "age": 35, "address": "123 Bedrock st, California 90210"},
  {"id": 2, "first_name": "Jane", "last_name": "Smith", "age": 27, "address": "234 Sagemaker drive, Texas 12345"},
  {"id": 3, "first_name": "Bob", "last_name": "Jones", "age": 42, "address": "111 DeepRacer ct, Virginia 55555"},
  {"id": 4, "first_name": "Sara", "last_name": "Miller", "age": 29, "address": "222 Robomaker ave, New Yotk 13579"},
  {"id": 5, "first_name": "Mark", "last_name": "Davis", "age": 31, "address": "444 Transcribe blvd, Florida 02468"},
  {"id": 6, "first_name": "Laura", "last_name": "Wilson", "age": 24, "address": "555 CodeGuru st, California 98765" },
  {"id": 7, "first_name": "Steve", "last_name": "Moore", "age": 36, "address": "456 DeepLens st, Texas 11223"},
  {"id": 8, "first_name": "Michelle", "last_name": "Chen", "age": 22, "address": "642 DeepCompose st, Colorado 33215"},
  {"id": 9, "first_name": "David", "last_name": "Lee", "age": 29, "address": "777 S3 st, California 99567"},
  {"id": 10, "first_name": "Jessica", "last_name": "Brown", "age": 18, "address": "909 Ec st, Utah 43210"}]

def address_lookup(id):
    for customer in customer_table:
        if customer["id"] == int(id):
            return customer
        
    return None

print(address_lookup(customer_id)["address"])

234 Sagemaker drive, Texas 12345
